<h3> Extracting protein sequences' features using Albert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
import torch
from transformers import AlbertModel, AlbertTokenizer, pipeline
import re
import numpy as np

<b>2. Set the file location of Albert and the vocabulary file<b>

In [2]:
modelPath = '/media/agemagician/Disk2/projects/bio_google_tpu_project/ahmed/models_pytorch/albert'
vocabPath = '/media/agemagician/Disk2/projects/bio_google_tpu_project/ahmed/models_pytorch/albert/albert_vocab_model.model'

<b>3. Load the vocabulary and Albert Model<b>

In [3]:
vocab = AlbertTokenizer(vocabPath, do_lower_case=False )

In [4]:
model = AlbertModel.from_pretrained(modelPath)

<b>4. Load the model into the GPU if avilabile and switch to inference mode<b>

In [5]:
model = pipeline('feature-extraction', model=model, tokenizer=vocab,device=0)

<b>5. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [6]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [7]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>6. Extracting sequences' features and covert the output to numpy if needed<b>

In [8]:
embedding = model(sequences_Example)

In [9]:
embedding = np.array(embedding)

In [10]:
print(embedding)

[[[-0.05933219 -0.00771756 -0.07200909 ... -0.05928943 -0.00733217
    0.16563264]
  [-0.06913061  0.03206101 -0.05003147 ... -0.11457862 -0.03595566
    0.08152848]
  [-0.33643803 -0.0175079   0.10670088 ... -0.20067249  0.17586312
    0.15829395]
  ...
  [-0.11349688 -0.02344881 -0.02433204 ... -0.19947116 -0.01309577
    0.06546066]
  [-0.08537761 -0.19285612 -0.01876267 ... -0.08882213  0.07482602
   -0.19948626]
  [-0.09240548 -0.00960725 -0.078479   ... -0.07284705 -0.0065897
    0.14899774]]

 [[-0.08919544  0.08583463 -0.03715046 ... -0.07874808 -0.00812221
    0.18468969]
  [-0.12822625  0.10716017 -0.01129915 ... -0.11677967  0.03904792
    0.10010004]
  [-0.36793104  0.03902491 -0.1156951  ... -0.2210128   0.10926703
    0.13324448]
  ...
  [-0.27154061  0.04539045 -0.05047561 ... -0.23157424  0.05315413
    0.14207189]
  [-0.18230185  0.10740501 -0.01663142 ... -0.23484249  0.06747213
    0.10624727]
  [-0.2350475  -0.03588062 -0.05941954 ... -0.24241649  0.05867656
    0.0

<b>Optional: Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by Albert model<b>

In [11]:
features = [] 

for seq_num in range(len(embedding)):
    seq_len = len(sequences_Example[seq_num].replace(" ", ""))
    start_Idx = 1
    end_Idx = seq_len+1
    seq_emd = embedding[seq_num][start_Idx:end_Idx]
    features.append(seq_emd)

In [12]:
print(features)

[array([[-0.06913061,  0.03206101, -0.05003147, ..., -0.11457862,
        -0.03595566,  0.08152848],
       [-0.33643803, -0.0175079 ,  0.10670088, ..., -0.20067249,
         0.17586312,  0.15829395],
       [-0.17356758,  0.03067275, -0.09666996, ..., -0.1891188 ,
         0.10043841,  0.14909169],
       ...,
       [-0.11075828, -0.19667947, -0.02256219, ..., -0.10256966,
         0.08064437, -0.21501049],
       [-0.11349688, -0.02344881, -0.02433204, ..., -0.19947116,
        -0.01309577,  0.06546066],
       [-0.08537761, -0.19285612, -0.01876267, ..., -0.08882213,
         0.07482602, -0.19948626]]), array([[-0.12822625,  0.10716017, -0.01129915, ..., -0.11677967,
         0.03904792,  0.10010004],
       [-0.36793104,  0.03902491, -0.1156951 , ..., -0.2210128 ,
         0.10926703,  0.13324448],
       [-0.26249766,  0.11969337, -0.05417463, ..., -0.20998038,
         0.11928435,  0.21757801],
       [-0.03641748, -0.08373182,  0.05713482, ..., -0.12186566,
         0.07572395,